**Title: NeuroWell AI**

**Libaries Import**

In [9]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [1]:
!pip install kaggle



**Data Set Gathering and Processing**

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nidhiroh","key":"744152cca65cbb29ad2e6d02816971f4"}'}

In [3]:
import shutil
import os

# Move kaggle.json to the correct directory
os.makedirs('/root/.kaggle', exist_ok=True)
shutil.move('kaggle.json', '/root/.kaggle/')

# Set permissions
os.chmod('/root/.kaggle/kaggle.json', 600)


In [4]:
!kaggle datasets download -d mamun1113/doctors-handwritten-prescription-bd-dataset

Dataset URL: https://www.kaggle.com/datasets/mamun1113/doctors-handwritten-prescription-bd-dataset
License(s): ODbL-1.0
 42% 8.00M/19.1M [00:00<00:00, 81.8MB/s]
100% 19.1M/19.1M [00:00<00:00, 128MB/s] 


In [5]:
import zipfile

with zipfile.ZipFile("doctors-handwritten-prescription-bd-dataset.zip", "r") as zip_ref:
    zip_ref.extractall("doctors-handwritten-prescription-bd-dataset")  # Extract to a folder


In [10]:
# Define the paths to the images and labels
image_folder = '/content/doctors-handwritten-prescription-bd-dataset/Doctor’s Handwritten Prescription BD dataset/Training/training_words'
label_file = '/content/doctors-handwritten-prescription-bd-dataset/Doctor’s Handwritten Prescription BD dataset/Training/training_labels.csv'

# Load labels
labels_df = pd.read_csv(label_file)

# Print column names to verify
print("Column names in the CSV file:", labels_df.columns)

# Initialize lists for images and labels
images = []
labels = []

# Correct the column names based on the printed output
filename_col = 'IMAGE'  # Replace with actual column name if different
label_col = 'MEDICINE_NAME'        # Replace with actual column name if different

# Debug: print the first few rows to understand the structure
print("First few rows of the dataframe:")
print(labels_df.head())

# Check if the specified columns exist
if filename_col not in labels_df.columns or label_col not in labels_df.columns:
    raise ValueError(f"Specified columns '{filename_col}' or '{label_col}' not found in the CSV file.")

# Load images and their corresponding labels
for index, row in labels_df.iterrows():
    img_path = os.path.join(image_folder, row[filename_col])
    img = load_img(img_path, target_size=(64, 64))  # Resize to a fixed size
    img = img_to_array(img)
    images.append(img)
    labels.append(row[label_col])

# Convert lists to numpy arrays
images = np.array(images, dtype="float") / 255.0  # Normalize pixel values
labels = np.array(labels)


Column names in the CSV file: Index(['IMAGE', 'MEDICINE_NAME', 'GENERIC_NAME'], dtype='object')
First few rows of the dataframe:
   IMAGE MEDICINE_NAME GENERIC_NAME
0  0.png         Aceta  Paracetamol
1  1.png         Aceta  Paracetamol
2  2.png         Aceta  Paracetamol
3  3.png         Aceta  Paracetamol
4  4.png         Aceta  Paracetamol


In [11]:
# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded, num_classes=78)  # 78 output classes

**Train-Test Split**

In [12]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(images, labels_categorical, test_size=0.2, random_state=42)

**Model Development**

In [15]:
# Build the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(78, activation='softmax'))  # 78 output classes

# Compile the model
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 24s 277ms/step - accuracy: 0.0128 - loss: 4.3806 - val_accuracy: 0.0048 - val_loss: 4.3590
Epoch 2/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 39s 261ms/step - accuracy: 0.0069 - loss: 4.3571 - val_accuracy: 0.0048 - val_loss: 4.3625
Epoch 3/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 19s 245ms/step - accuracy: 0.0175 - loss: 4.3426 - val_accuracy: 0.0208 - val_loss: 4.2473
Epoch 4/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 22s 270ms/step - accuracy: 0.0451 - loss: 4.1565 - val_accuracy: 0.0801 - val_loss: 3.9137
Epoch 5/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 39s 243ms/step - accuracy: 0.1325 - loss: 3.6313 - val_accuracy: 0.3221 - val_loss: 2.9751
Epoch 6/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 23s 273ms/step - accuracy: 0.3543 - loss: 2.5530 - val_accuracy: 0.5353 - val_loss: 2.0295
Epoch 7/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 40s 258ms/step - accuracy: 0.5651 - loss: 1.6900 - val_accuracy: 0.5913 - val_loss: 1.5741
Epoch 8/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 19s 241ms/step - accuracy: 0.6921 - loss: 1.1950 - val_accu

**Performance Check**

In [16]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 101ms/step - accuracy: 0.7221 - loss: 1.1467
Test Accuracy: 70.19%


**Saving the Model**

In [18]:
# Save the model
model.save('prescription_classification_model.keras')